# 0. Load Data, Library

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch torchaudio transformers datasets

In [3]:
!git clone https://github.com/PhamPham2S/NewJeans-5.git

fatal: destination path 'NewJeans-5' already exists and is not an empty directory.


In [4]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 1/Project")
SRC_ROOT = PROJECT_ROOT / "src"
if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))


In [5]:
%cd "/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 1/Project"

/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 1/Project


In [6]:
# reimport
import importlib
# importlib.reload(sys.modules['src.core.grad_monitor'])

# src/core 모듈 import
from src.core.data_pipeline import build_embedding_dataloader
from src.core.modeling import FusionModel_train as FusionModel
from src.core.modeling_cp import FusionModel_train as FusionModel_cp
from src.core.multitask import MultiTaskLoss, MultiTaskLossController
from src.core.grad_monitor import GradientMonitor
from src.core.losses import ordinal_loss

# 기타 PyTorch
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from pathlib import Path

# 1. Hyperparameter setting

In [7]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 8
EPOCHS = 2
LEARNING_RATE = 0.01 # 초기 시도 1e-5 -> 1e-4 -> 5e-4

# 2. Data Loader

In [8]:
# # Audio Embedding load
# from glob import glob
# from tqdm import tqdm

# # audio_root = Path("/content/drive/MyDrive/LikeLion/실전 프로젝트 1/Project/data/emb/aud_emb/embed-audio-hybrid")
# audio_root = Path("/content/NewJeans-5/data/emb/10000_aud_emb")

# audio_embeddings = []

# for npy_file in tqdm(audio_root.glob("*.npy")):
#     emb = np.load(npy_file)
#     audio_embeddings.append(
#         {
#             "emb":emb,
#             "id": npy_file.stem
#         })

# print(f"로드된 audio embedding 개수: {len(audio_embeddings)}")
# # print(f"예시 shape: {audio_embeddings[0].shape}")


In [9]:
# # Text Embedding load
# # txt_root = Path("/content/drive/MyDrive/LikeLion/실전 프로젝트 1/Project/data/emb/txt_emb")
# txt_root = Path("/content/NewJeans-5/data/emb/txt_emd_data")
# txt_embeddings = []

# for folder in txt_root.iterdir():
#     if folder.is_dir():
#         for npy_file in folder.iterdir():
#             if npy_file.is_file() and npy_file.suffix == ".npy":
#                 # print(npy_file)
#                 # print(npy_file.stem)
#                 emb = np.load(npy_file)   # 실제 npy 내부 데이터 로드
#                 txt_embeddings.append(
#                     {
#                         "emb":emb,
#                         "id": npy_file.stem
#                     })

# print(f"로드된 txt embedding 개수: {len(txt_embeddings)}")
# # print(f"예시 shape: {txt_embeddings[0].shape}")


In [10]:
# # Data Concat
# text_index = {
#     item["id"]: item["emb"]
#     for item in txt_embeddings
# }

# embeddings = {}

# for item in audio_embeddings:
#     id = item["id"]

#     if id not in text_index:
#         continue  # or log

#     embeddings[id] = {
#         "audio": item["emb"],
#         "text": text_index[id],
#     }

# print(f"합쳐진 embedding 총 개수: {len(embeddings)}")

# missing_text = [
#     item["id"] for item in audio_embeddings
#     if item["id"] not in text_index
# ]

# print("text missing:", len(missing_text))
# print(missing_text[:5])

In [11]:
!pip install tqdm

In [12]:
# # Labeling
# import json
# from tqdm import tqdm
# json_root = Path("/content/drive/MyDrive/LikeLion/실전 프로젝트 1/Project/data/emb/json_emb")

# def find_json_by_id(json_root: Path, id: str) -> Path | None:
#     """
#     여러 폴더 중에서 {id}.json 파일 경로 반환
#     """
#     for folder in json_root.iterdir():
#         # print(folder)
#         if not folder.is_dir():
#             continue

#         json_path = folder / f"{id}.json"
#         if json_path.exists():
#             # print(f"JSON 파일 존재 : {json_path}")
#             return json_path
#     print(f"해당하는 json 파일 없음 : {json_path}")
#     return None

# for id in tqdm(embeddings.keys()):
#     json_path = find_json_by_id(json_root, id)

#     if json_path is None:
#         continue  # or log warning

#     with open(json_path, "r", encoding="utf-8") as f:
#         data = json.load(f)

#     embeddings[id]["urgency"] = data["urgencyLevel"]
#     embeddings[id]["sentiment"] = data["sentiment"]

#     # print(f"완료된 id: {id}")

In [13]:
# # 최종 임베딩 파일 저장 - total_emb.pkl
# import pickle

# with open("/content/drive/MyDrive/LikeLion/실전 프로젝트 1/Project/data/emb/total_emb.pkl", "wb") as f:
#     pickle.dump(embeddings, f)

In [14]:
import pickle

In [15]:
with open("/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 1/Project/data/emb/total_emb.pkl", "rb") as f:
    embeddings = pickle.load(f)

URGENCY_ORDER = dict({"상":0, "중":1, "하":2})
SENTIMENT_ORDER = dict({"당황/난처":0, "불안/걱정":1, "중립":2, "기타부정":3})

train_loader = build_embedding_dataloader(
    audio_embeds=[v["audio"] for v in embeddings.values()],
    text_embeds=[v["text"] for v in embeddings.values()],
    urgencies=[URGENCY_ORDER[v["urgency"]] for v in embeddings.values()],
    sentiments=[SENTIMENT_ORDER[v["sentiment"]] for v in embeddings.values()],
    batch_size=BATCH_SIZE,
    shuffle=True,
)

In [16]:
URGENCY_ORDER = dict({"상":0, "중":1, "하":2})
URGENCY_ORDER

{'상': 0, '중': 1, '하': 2}

# 3. Model, Multitask Loss Initialize

In [17]:
import importlib
import src.core.modeling_cp as modeling_cp
importlib.reload(modeling_cp)

<module 'src.core.modeling_cp' from '/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 1/Project/src/core/modeling_cp.py'>

In [18]:
# Model
model = FusionModel(
    urgency_levels=3,
    sentiment_levels=4,
    fusion_dim=256,
    dropout=0.2
).to(DEVICE)

# MultiTask Loss
controller = MultiTaskLossController(
    warmup_epochs=1,
    urgency_weight=1.0,
    sentiment_weight=1.0,
    use_uncertainty=False
)

criterion = MultiTaskLoss(
    urgency_loss_fn=ordinal_loss,
    sentiment_loss_fn=nn.CrossEntropyLoss(),
    controller=controller
)

# Audio Encoder freeze
# for param in model.audio_encoder.parameters():
#     param.requires_grad = False

optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

grad_monitor = GradientMonitor(model)



In [19]:
import importlib
import src.core.modeling_cp as modeling_cp
importlib.reload(modeling_cp)

model_cp = modeling_cp.FusionModel_train(
    urgency_levels=3,
    sentiment_levels=4,
    fusion_dim=256,
    dropout=0.2
).to(DEVICE)

print(model_cp)


FusionModel_train(
  (fusion): Sequential(
    (0): LayerNorm((1549,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=1549, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.2, inplace=False)
  )
  (urgency_head): Linear(in_features=256, out_features=2, bias=True)
  (sentiment_head): Linear(in_features=256, out_features=4, bias=True)
)


In [20]:

grad_monitor_cp = GradientMonitor(model_cp) # model 바꾸게 되면 건드리는 코드!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [21]:
for i, layer in enumerate(model_cp.fusion):
    print(i, layer)


0 LayerNorm((1549,), eps=1e-05, elementwise_affine=True)
1 Linear(in_features=1549, out_features=256, bias=True)
2 ReLU()
3 Dropout(p=0.2, inplace=False)
4 Linear(in_features=256, out_features=256, bias=True)
5 ReLU()
6 Dropout(p=0.2, inplace=False)


# 4. Training Loop

In [22]:
import torch
torch.cuda.empty_cache()

In [23]:
from tqdm import tqdm

In [24]:
for epoch in tqdm(range(EPOCHS)):
    model_cp.train() # model 바꾸게 되면 건드리는 코드!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    for batch_idx, batch in enumerate(train_loader):
        # batch GPU로 이동
        for k in batch:
            batch[k] = batch[k].to(DEVICE)

        optimizer.zero_grad()

        # forward
        outputs = model_cp(batch) # model 바꾸게 되면 건드리는 코드!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        outputs = {
            "urgency": outputs["urgency"],
            "sentiment": outputs["sentiment"],
        }

        # targets reshape
        targets = {
            "urgency": batch["urgency"].view(-1).float(),
            "sentiment": batch["sentiment"].view(-1).long()
        }

        # loss 계산
        losses = criterion(outputs, targets, epoch)

        # 1. gradient 관찰 (graph 살아 있음)
        grad_stats = grad_monitor_cp.log(
            {
                "urgency": losses["urgency"],
                "sentiment": losses["sentiment"],
            },
            step=batch_idx,
            epoch=epoch,
        )

        # 2. 실제 학습
        losses["total"].backward()
        optimizer.step()


        if batch_idx % 100 == 0:
            print(grad_stats)


        if batch_idx % 50 == 0:
            print(f"Epoch [{epoch+1}/{EPOCHS}], Batch {batch_idx}, Loss: {losses["total"].item():.4f}")


  0%|          | 0/2 [00:00<?, ?it/s]

{'urgency_grad_norm': 1.4430325031280518, 'sentiment_grad_norm': 3.8853728771209717, 'grad_cosine_similarity': 0.021410847082734108}
Epoch [1/2], Batch 0, Loss: 0.6866
Epoch [1/2], Batch 50, Loss: 0.7269
{'urgency_grad_norm': 1.676779866218567, 'sentiment_grad_norm': 2.9182183742523193, 'grad_cosine_similarity': -0.08978565037250519}
Epoch [1/2], Batch 100, Loss: 0.7105
Epoch [1/2], Batch 150, Loss: 0.7183
{'urgency_grad_norm': 1.3946012258529663, 'sentiment_grad_norm': 3.353699207305908, 'grad_cosine_similarity': -0.07048600912094116}
Epoch [1/2], Batch 200, Loss: 0.6746
Epoch [1/2], Batch 250, Loss: 0.7072
{'urgency_grad_norm': 1.6857514381408691, 'sentiment_grad_norm': 3.4648587703704834, 'grad_cosine_similarity': -0.012537717819213867}
Epoch [1/2], Batch 300, Loss: 0.7249
Epoch [1/2], Batch 350, Loss: 0.6743
{'urgency_grad_norm': 1.2512317895889282, 'sentiment_grad_norm': 3.451197385787964, 'grad_cosine_similarity': 0.058627694845199585}
Epoch [1/2], Batch 400, Loss: 0.7182
Epoch [

 50%|█████     | 1/2 [00:05<00:05,  5.20s/it]

{'urgency_grad_norm': 1.0821727514266968, 'sentiment_grad_norm': 4.033103942871094, 'grad_cosine_similarity': -0.011043645441532135}
Epoch [1/2], Batch 1200, Loss: 0.6977
{'urgency_grad_norm': 1.2115116119384766, 'sentiment_grad_norm': 2.5527608394622803, 'grad_cosine_similarity': -0.08267064392566681}
Epoch [2/2], Batch 0, Loss: 2.0787
Epoch [2/2], Batch 50, Loss: 2.0657
{'urgency_grad_norm': 1.564234972000122, 'sentiment_grad_norm': 2.7691192626953125, 'grad_cosine_similarity': -0.1355568915605545}
Epoch [2/2], Batch 100, Loss: 2.0873
Epoch [2/2], Batch 150, Loss: 2.0770
{'urgency_grad_norm': 1.2760227918624878, 'sentiment_grad_norm': 2.8626933097839355, 'grad_cosine_similarity': -0.1354423612356186}
Epoch [2/2], Batch 200, Loss: 2.0421
Epoch [2/2], Batch 250, Loss: 2.0774
{'urgency_grad_norm': 1.30221688747406, 'sentiment_grad_norm': 3.1787304878234863, 'grad_cosine_similarity': -0.049442730844020844}
Epoch [2/2], Batch 300, Loss: 2.1213
Epoch [2/2], Batch 350, Loss: 2.0838
{'urgenc

100%|██████████| 2/2 [00:10<00:00,  5.14s/it]


In [26]:
print(batch['urgency'].shape, batch['urgency'].dtype)
print(batch['sentiment'].shape, batch['sentiment'].dtype)
print(outputs['urgency'].shape, outputs['sentiment'].shape)

torch.Size([8]) torch.int64
torch.Size([8]) torch.int64
torch.Size([8, 2]) torch.Size([8, 4])


In [25]:
import matplotlib.pyplot as plt

epochs = range(len(history["urgency_loss"]))

plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(epochs, history["urgency_loss"], label="Urgency Loss")
plt.plot(epochs, history["sentiment_loss"], label="Sentiment Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Task-wise Loss")
plt.legend()

# Gradient norm plot
plt.subplot(1, 2, 2)
plt.plot(epochs, history["urgency_grad_norm"], label="Urgency Grad Norm")
plt.plot(epochs, history["sentiment_grad_norm"], label="Sentiment Grad Norm")
plt.xlabel("Epoch")
plt.ylabel("Gradient Norm")
plt.title("Task-wise Gradient Norm")
plt.legend()

plt.tight_layout()
plt.show()


NameError: name 'history' is not defined

In [ ]:
# Model Save

MODEL_DIR =

# fusion layer
torch.save(
    model.fusion.state_dict(),
    "fusion_linear.pt"
)

# urgency head
torch.save(
    model.urgency_head.state_dict(),
    "urgency_head.pt"
)

# sentiment head
torch.save(
    model.sentiment_head.state_dict(),
    "sentiment_head.pt"
)
